# Word embedding models

In [3]:
from koselleck import *

## Cache existing data in db

In [4]:
# save_vecs_in_db(num_proc=1)

In [5]:
vl=get_veclib('vecs')

In [9]:
# list(vl.keys())

NameError: name 'dfmodels' is not defined

In [32]:
get_vecs('1720-1725')

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,0.518834,0.112339,0.354481,1.326438,-0.350878,1.323567,-0.152572,0.241625,0.098877,-0.079031,...,-0.000588,-0.067636,1.035088,-0.258235,-0.470933,-0.608469,0.038985,0.250557,0.250846,0.047013
of,-0.352763,0.553724,-0.055193,1.560007,-0.436875,0.806038,-0.453449,1.287194,-1.133802,-0.261347,...,0.147312,-0.008487,1.155754,-1.370358,-0.499677,-0.171854,0.341440,-0.103620,-0.160002,0.178899
to,0.214600,0.678866,-0.436494,0.772427,0.170232,2.645017,0.343733,-1.033259,0.416046,0.296791,...,-0.810945,-0.645016,0.598870,-1.679335,-0.539725,-1.543595,0.039166,0.912559,1.334654,-2.360000
and,-0.744995,0.243157,0.385076,1.661384,1.101202,1.322934,-0.390144,0.393504,0.710245,-0.126497,...,-1.179631,-0.442142,0.615115,-1.465211,-0.630602,-0.440380,-0.159276,1.070248,-0.170253,0.250179
in,-0.441625,0.526975,0.996545,1.096613,-0.980460,0.987092,0.991870,0.590672,-1.330634,0.027227,...,-0.266627,-0.190203,1.840565,-1.020965,-0.100212,-1.303767,-0.322558,0.159598,-0.345350,-0.231719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
angelis,-0.019318,-0.084302,0.269294,0.113684,0.166686,0.046743,0.103498,0.018679,-0.129501,-0.184194,...,0.307812,0.246739,0.126733,-0.021164,0.025616,0.037745,-0.128690,0.147227,0.044234,-0.112729
pyrats,-0.078977,-0.116150,-0.007428,-0.307214,-0.003595,-0.048145,0.060895,0.073208,0.060787,-0.228901,...,0.016941,0.492123,0.133330,0.020916,0.386024,0.326242,0.366853,0.021573,-0.399174,0.015134
pastoral,-0.030481,0.006958,-0.040011,-0.077788,0.027891,-0.003338,-0.121442,-0.048619,-0.166454,0.192653,...,-0.004803,0.091070,-0.085037,-0.224614,-0.134571,0.191833,0.035105,-0.147031,0.109892,-0.020265
rattle,-0.073694,-0.003752,0.151062,0.150766,0.117401,-0.104978,0.091885,0.295049,0.025139,0.077934,...,0.157638,-0.194139,0.038654,-0.128295,0.054549,0.052017,-0.091437,0.116301,-0.019780,-0.199177


In [8]:
dbget('vec(virtue_1720-1725)')

In [ ]:
stop

## Generate data

### Create models

In [ ]:
def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25, calc_numlines=False):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])
    if calc_numlines:
        odf['num_lines']=odf.path.progress_apply(get_numlines)
        odf['num_words']=odf['num_lines']*skipgram_n
    return odf#.query('1680<=year<1970')

In [ ]:
# get_skipgrams()

In [ ]:
dfskip=get_skipgrams(calc_numlines=True)
# dfskip=dfskip.query('1720<=year<1900')
dfskip['period']=dfskip.year.apply(lambda y: periodize(y,YEARBIN))
dfskip

In [ ]:
# dfskip.groupby('period').num_words.sum()

In [ ]:
def get_dfskipruns(dfskip,num_runs=10,incl_existing=False):
    dfskipruns=pd.concat([
        dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
        for i in range(num_runs)
    ])
    dfskipruns['opath']=dfskipruns.apply(lambda row: os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run,'model.bin'),1)
    dfskipruns['opath_exists']=dfskipruns.opath.apply(lambda x: os.path.exists(x))
    if not incl_existing: dfskipruns=dfskipruns[dfskipruns.opath_exists==False]
    return dfskipruns

In [ ]:
# get_dfskipruns(dfskip,num_runs=2)

In [ ]:
def gen_and_save_model(dfskip,nskip=DEFAULT_NUM_SKIP,force=False,vector_size=100,window=10,min_count=5,epochs=10,workers=8,verbose=False):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        disable_gensim_logging() if not verbose else enable_gensim_logging()
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame([{'fnfn':ofnfn}])


In [ ]:
# fnfn=gen_and_save_model(get_dfskipruns(dfskip).iloc[:1], force=True).fnfn.iloc[0]
# load_model(fnfn).wv.most_similar('value')

In [ ]:
# res=pmap_groups(
#     gen_and_save_model,
#     dfskipruns.groupby(['period','run']),
#     num_proc=4,
#     kwargs=dict(force=True, nskip=NSKIP_PER_YR)
# )

In [ ]:
def gen_models(
        ybin=5,
        ymin=1680,
        ymax=1970,
        num_runs=1,
        force=False,
        nskip_per_yr=NSKIP_PER_YR
    ):
    dfskip=get_skipgrams(calc_numlines=False).query(f'{ymin}<=year<{ymax}')
    dfskip['period']=dfskip.year.apply(lambda y: periodize(y,ybin))
    dfskipruns=get_dfskipruns(dfskip, num_runs=num_runs, incl_existing=force)
    dfgrps=dfskipruns.groupby(['period','run'])
    print(f'Generating {len(dfgrps)} new models over {dfskipruns.period.nunique()} periods and {dfskipruns.run.nunique()} runs')
    return pmap_groups(
        gen_and_save_model,
        dfskipruns.groupby(['period','run']),
        num_proc=4,
        kwargs=dict(force=force, nskip=nskip_per_yr)
    )

In [ ]:
gen_models(num_runs=10)

## Test models

In [ ]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1805-1810/run_25/model.bin')
m.wv.most_similar('virtue')

In [ ]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1965-1970/run_07/model.bin')
m.wv.most_similar(['king','woman'],['man'])

In [ ]:
def test_models(dfmodels,gby=['period','run']):
    o=[]
    dfgrp=dfmodels.groupby(gby)
    for period,dfg in tqdm(sorted(dfgrp)):#, total=len(dfgrp)):
        path=dfg.iloc[-1].path
        m=load_model(path)
        try:
            testvec=m.wv.most_similar(['king','woman'],['man'],topn=25)
        except KeyError:
            continue
        testvec_wl=[x for x,y in testvec]
        has_queen='queen' in set(testvec_wl)
        odx={
            **dict(zip(gby,period)),
            'has_queen':has_queen,
            'rank_queen':testvec_wl.index('queen') if has_queen else np.nan,
            'neighborhood':', '.join(testvec_wl),
        }
        o+=[odx]
#         break
    return pd.DataFrame(o)

In [ ]:
dfmodels = get_pathdf_models().query('period_len==5')
dftests  = test_models(dfmodels)
dftests

In [ ]:
dftests.to_csv('../../data/data.model.tests.csv')

In [ ]:
dftests.query('has_queen==True').groupby('period').size()